In [1]:
import casadi
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from matplotlib.animation import FuncAnimation

In [2]:
g = 9.8  # m/s^2

# https://en.wikipedia.org/wiki/SpaceX_Super_Heavy
m = 3_675_000  # kg
L = 69  # m
I = 1 / 12 * m * L**2  # https://en.wikipedia.org/wiki/List_of_moments_of_inertia  == thin rod rotating around its COM

# https://en.wikipedia.org/wiki/SpaceX_Super_Heavy
maximum_thrust = 69.9 * 1e6  # N

max_alpha = np.deg2rad(15)
min_alpha = -max_alpha

# Define optimization variables
N = 400  # Number of control intervals
dt = 0.05
T = N * dt

# define dynamics
nq = 3  # x position, y position, theta
nv = 3  # x velocity, y velocity, angular velocity
nu = 2  # thrust, alpha

cq = casadi.SX.sym("q", nq)  # generalized position
cv = casadi.SX.sym("v", nv)  # generalized velocity
cu = casadi.SX.sym("u", nu)  # control input

Fx = maximum_thrust * cu[0] * casadi.sin(cu[1] + cq[2])
Fy = maximum_thrust * cu[0] * casadi.cos(cu[1] + cq[2])
T = -maximum_thrust * cu[0] * L / 2 * casadi.sin(cu[1])

nv0_dot = Fx / m
nv1_dot = Fy / m - g
nv2_dot = T / I

v_dot = casadi.vertcat(nv0_dot, nv1_dot, nv2_dot)
aba_fn = casadi.Function("aba_fn", [cq, cv, cu], [v_dot])


# Euler integrator
def euler_integrate(q, v, u):
    q_next = q + v * dt
    v_next = v + aba_fn(q, v, u) * dt
    return q_next, v_next

In [ ]:
opti = casadi.Opti()

Q = opti.variable(nq, N + 1)
V = opti.variable(nv, N + 1)
U = opti.variable(nu, N)


# Set initial state
opti.subject_to(Q[:, 0] == np.array([-400, 1000, -np.pi / 5]))
opti.subject_to(V[:, 0] == np.array([90, -80, 0]))

# Set final state
opti.subject_to(Q[:, N] == np.array([0, 0, 0]))
opti.subject_to(V[:, N] == np.array([0, 0, 0]))

# Set dynamics constraints
for k in range(N):
    q_next, v_next = euler_integrate(
        Q[:, k],
        V[:, k],
        U[:, k],
    )
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

# Set control limits
for i in range(N - 1):
    lower, upper = 0.4, 1.0
    opti.subject_to(opti.bounded(lower, U[0, i], upper))
    opti.subject_to(opti.bounded(min_alpha, U[1, i], max_alpha))

# Define objective
obj = 0
for k in range(N):
    obj += U[:, k].T @ U[:, k]
    obj += 5 * Q[2, k] ** 2
    if k != N - 1:
        obj += 10 * (U[:, k] - U[:, k + 1]).T @ (U[:, k] - U[:, k + 1])
        obj += 10 * (Q[2, k] - Q[2, k + 1]) ** 2

    if k >= 0.9 * N:
        obj += 300 * Q[2, k] ** 2

opti.minimize(obj)

# Solve
opti.solver("ipopt")
sol = opti.solve()

In [ ]:
Q_traj = sol.value(Q)
U_traj = sol.value(U)

fig = plt.figure(figsize=(5, 5), constrained_layout=False)

ax1 = fig.add_subplot(111)
(ln6,) = ax1.plot([], [], "-.", linewidth=2, color="orange")
(ln2,) = ax1.plot([], [], linewidth=2, color="tomato")
(ln1,) = ax1.plot([], [], linewidth=5, color="lightblue")

# plt.axis('off')
plt.tight_layout()

ax1.set_xlim(-400, 400)
ax1.set_ylim(-50, 1000)
ax1.set_aspect(1)


def update(i):
    x, y, theta = Q_traj[:, i]

    x_points = [x + L / 2 * np.sin(theta), x - L / 2 * np.sin(theta)]
    y_points = [y + L / 2 * np.cos(theta), y - L / 2 * np.cos(theta)]
    ln1.set_data(x_points, y_points)

    thrust, alpha = U_traj[:, i]
    flame_length = 60 * thrust

    flame_x = [x_points[1], x_points[1] - flame_length * np.sin(theta + alpha)]
    flame_y = [y_points[1], y_points[1] - flame_length * np.cos(theta + alpha)]

    ln2.set_data(flame_x, flame_y)

    ln6.set_data(Q_traj[0, :i], Q_traj[1, :i])


anim = FuncAnimation(fig, update, range(N), interval=dt * 1000)

print("Generating animation...")
plt.close()
html = HTML(anim.to_jshtml())
print("Done!")
display(html)

In [ ]:
display(html)

In [ ]:
fontsize = 25
storefig = True

# Create a figure with multiple snapshots of the trajectory
fig, axes = plt.subplots(2, 2, figsize=(20, 20))
axes = axes.flatten()  # Flatten 2D array to 1D for easier iteration
snapshot_indices = np.linspace(0, N-1, 4, dtype=int)

for ax, i in zip(axes, snapshot_indices):
    x, y, theta = Q_traj[:, i]
    
    # Draw rocket body
    x_points = [x + L/2 * np.sin(theta), x - L/2 * np.sin(theta)]
    y_points = [y + L/2 * np.cos(theta), y - L/2 * np.cos(theta)]
    
    # Draw thrust flame
    thrust, alpha = U_traj[:, i]
    flame_length = 60 * thrust
    flame_x = [x_points[1], x_points[1] - flame_length * np.sin(theta + alpha)]
    flame_y = [y_points[1], y_points[1] - flame_length * np.cos(theta + alpha)]
    ax.plot(flame_x, flame_y, linewidth=6, color='tomato')
    ax.plot(x_points, y_points, linewidth=12, color='lightblue')
    
    # Draw trajectory up to this point
    ax.plot(Q_traj[0, :i], Q_traj[1, :i], '-.', linewidth=2, color='orange', label='Trajectory')
    
    # Set consistent view limits and aspect ratio
    ax.set_xlim(-400, 400)
    ax.set_ylim(-50, 1000)
    ax.set_aspect(1)
    ax.set_title(f't = {i*dt:.1f}s', fontsize=fontsize)
    
    # Set font size for axis labels and ticks
    ax.tick_params(axis='both', which='major', labelsize=fontsize)
    
    # Add legend
    ax.legend(fontsize=fontsize)

plt.tight_layout()
if storefig:
    plt.savefig("starship.png", dpi=600)
plt.show()
